# County Demographics
---

### Why use a separate dataset for population?
- Most population groups are present in the census data (gender, age, etc.) So why use an additional population dataset instead?
  - The population dataset claims to provide "actual" numbers, whereas the census data claims to provide "estimates"
  - The population dataset is more precise, with age groups of each individual age number, allowing us to make our own aggregated bins (adult, minor). The census data has age groups defined already, but in increments of 5, so the middle group is "15 to 19", but we need 18 and under!
  - The population dataset offers sub-aggregations: we have `minor_female` and `minor_male`, for instance, whereas the census data only offers age populations and gender populations separately
- So instead, we will use population dataset first, and add in additional groups from census data

In [14]:
%run 1-workspace_setup.ipynb

INDEX = ['year', 'county']

## Population

> This data will supplement our census data in the next step of data prep. We're using the Population dataset because it's more accurate (census is just estimates), and it lets us create the age grouping ourselves. There are nearly 400,000 rows, because they give us population by year, county, and EACH individual age. In our case, we want to create an age grouping that separates students in school, and adults. So we chose >= 19, and < 19. It also has a 60 year timeframe. So number of rows = 60 years * 64 counties * 90 years of age.

In [15]:
df_raw = read_raw("""
    year,
    UPPER(county) AS county,
    age,
    malePopulation AS male,
    femalePopulation AS female,
    totalPopulation AS total
""",
FROM="county_population"
)
head(df_raw)

6 cols x 381504 rows


,year,county,age,male,female,total
0,1990,ADAMS,0,2354,2404,4758
1,1990,ADAMS,1,2345,2375,4720
2,1990,ADAMS,2,2413,2219,4632


### Age groups (< 19, >= 19)

In [16]:
df = df_raw.coerce_type(float, exclude=['year'])
df['age_range'] = "over18"
df.loc[df.age <= 18, 'age_range'] = 'under19'
df = df.drop_cols('age')
head(df)

df = (df
    .groupby(INDEX + ['age_range'])
    .sum()
    .reset_index()
)
df_grouped = df
head(df_grouped)

6 cols x 381504 rows


,year,county,male,female,total,age_range
0,1990,ADAMS,2354.0,2404.0,4758.0,under19
1,1990,ADAMS,2345.0,2375.0,4720.0,under19
2,1990,ADAMS,2413.0,2219.0,4632.0,under19


6 cols x 7808 rows


,year,county,age_range,male,female,total
0,1990,ADAMS,over18,90383.0,94282.0,184665.0
1,1990,ADAMS,under19,41519.0,39525.0,81044.0
2,1990,ALAMOSA,over18,4488.0,4823.0,9311.0


### Notice the `age_range` column. We should pivot those values out to their own columns, and mix with our existing columns
- First, pivot age_range into the male, female, and total columns
- We're left with a multilevel column index, so we drop a level and rename everything by hand.
- Lastly, restore the total, male, and female columns since they got split in half when pivoting.

In [17]:
df = (df_grouped
    .pivot(
        index=INDEX,
        columns='age_range',
        values=['male', 'female', 'total']
    )
    .reset_multilevel_columns(INDEX + ['over18', 'under19', 'under19_male', 'under19_female', 'over18_male', 'over18_female'])
)

df = (df
    .insert_at(2, 'female', df.under19_female + df.over18_female)
    .insert_at(2, 'male', df.under19_male + df.over18_male)
    .insert_at(2, 'total', df.under19 + df.over18)
)

pop_raw = df
head(pop_raw)

11 cols x 3904 rows


,year,county,total,male,female,over18,under19,under19_male,under19_female,over18_male,over18_female
0,1990,ADAMS,131902.0,278947.0,120569.0,90383.0,41519.0,94282.0,39525.0,184665.0,81044.0
1,1990,ALAMOSA,6677.0,14134.0,6423.0,4488.0,2189.0,4823.0,2117.0,9311.0,4306.0
2,1990,ARAPAHOE,191722.0,428121.0,166735.0,134481.0,57241.0,146820.0,54747.0,281301.0,111988.0


In [18]:
write_main(pop_raw, 'county_population')

3904

---
---
---

## Census Field Descriptions
---
- To supplement the "Census Counties ..." datasets, they've provided us a table with descriptions of each column name, for each historical standard of the census. Fortunately, the 2019 and 2012 census data (that's what we're using) uses the same standard: `acs_standard`
- This script does the following:
  - Filters the source dataframe to only include `acs_standard` column descriptions
  - Selects only necessary columns (column name, description)
  - Renames some values in column name, and removes some column name values we'll never use (geonum, geojson)

In [19]:
desc = read_raw("""
        apifieldname AS field_name,
        description
    """,
    FROM = "census_counties_field_desc",
    WHERE = "type = 'acs_standard' AND field_name NOT IN ('geonum', 'geojson')"
)
desc.loc[desc.field_name == 'geoname', 'field_name'] = 'county'

write_main(desc, 'census_counties_field_desc')

head(desc)

2 cols x 155 rows


,field_name,description
0,county,Geographic Area common name
1,pop,Population Estimate for the given time range
2,hispanic,Estimate for the Hispanic Population


---
---
---

# Engineering Census data
- Prepare census data for a variety of uses, such as visualization, aggregation and predictive modeling

In [20]:
# Each census year comes in a separate dataset
dem = (
    pd.concat([
            (read_raw(f'census_counties_{year}')
                .drop_cols('pop', 'geonum', 'the_geom')
                .assign(year=year)
                .rename_col('civ_ni_','civ_ni_p')
            )
        for year in range(2012, 2020)
    ])
    .copy() # avoid fragmentation caused by assign()
    .rename_col('geoname','county')
    .move_col('year', 0)
    .coerce_type(float, exclude='year')
)

dem.county = (
    dem.county
    .str.replace(" County, Colorado", "")
    .str.upper()
)
head(dem)

155 cols x 512 rows


,year,county,hispanic,white_nh,black_nh,ntvam_nh,asian_nh,hawpi_nh,other_nh,twoplus_nh,...,civ_ni_pop,disabled,pop16_pls,laborforce,civ_lf,emp,unemp,armedfrcs,not_lf,civ_ni_p
0,2012,ARAPAHOE,105174.0,364766.0,55629.0,2211.0,28067.0,1166.0,1267.0,16077.0,...,568663.0,49870.0,444215.0,320199.0,318041.0,292089.0,25952.0,2158.0,124016.0,568663.0
1,2012,MINERAL,15.0,671.0,9.0,5.0,1.0,0.0,0.0,1.0,...,702.0,129.0,681.0,391.0,391.0,370.0,21.0,0.0,290.0,702.0
2,2012,MONTROSE,8037.0,31799.0,186.0,74.0,227.0,49.0,33.0,589.0,...,40552.0,5649.0,32334.0,20137.0,20124.0,18110.0,2014.0,13.0,12197.0,40552.0


#### Select desired columns from census data

In [21]:
df = dem.copy()[INDEX + [
    'med_age',
    'households', 'avghhsize',
    'civ_lf', 'emp', 'unemp',
    'hispanic', 'white_nh', 'black_nh', 'asian_nh', 'ntvam_nh', 'hawpi_nh', 'other_nh', 'twoplus_nh',
    'pop25plus', 'hsgrad_sc',
    'med_hh_inc', 'per_cap_in',
    'citz_birth', 'citz_nat', 'born_in_co',
    'pop_3pl', 'enrolled', 'undergrad',
    'gr_1_4', 'gr_5_8', 'gr_9_12',
    'med_hm_val', 'med_yr_blt',
    'housing_un', 'occ_hu',
    'own_occ_hu', 'v_l_50k', 'v50k_100k', 'v100k_150k', 'v150k_200k', 'v200k_250k', 'v250k_300k',
    'v300k_400k', 'v400k_500k', 'v500k_750k', 'v750k_1m', 'v_1m_plus',
    'b2000_2009', 'b1990_1999', 'b1980_1989', 'b1970_1979',
    'b1960_1969', 'b1950_1959', 'b1940_1949', 'b1939_e',
    'ps_uni', 'ps_below',
    'tot_l18', 'pov_l18',
]]

#### Group bins together

In [22]:
from typing import Callable
@Extend(pd.DataFrame)
def combine_cols(
    self,
    name: str | None = None,
    cols: list | None = None,
    items: dict | None = None,
    replace: bool = True,
    func: Callable[[Iterable[Any]], int | float] = sum
) -> pd.DataFrame:
    """
    Given a list of column names, create a new column with their sum, and
    position it before the first col in 'cols'. So if replace=True, then
    the old columns will effectively be replaced in their original position.
    To do multiple sums, pass 'items' as dict with names as keys and col list as vals
    """
    df = self.copy()
    if not items:
        items = {name: cols}
    for name, cols in items.items():
        new = func([df[c] for c in cols])
        df = df.insert_at(cols[0], name, new)
        if replace:
            df = df.drop_cols(cols)
    return df

In [23]:
# Create new variable for total citizens. Place it next to citz_birth
df = (df
    .insert_at('citz_birth', 'citz', df.citz_birth + df.citz_nat)
    .drop_cols('citz_nat')
    .combine_cols(items={
        'race_other': ['ntvam_nh', 'hawpi_nh', 'other_nh', 'twoplus_nh'],
        'b1949_e': ['b1939_e', 'b1940_1949'],
        'v50k_150k':  ['v50k_100k', 'v100k_150k'],
        'v150k_250k': ['v150k_200k', 'v200k_250k'],
        'v250k_400k': ['v250k_300k', 'v300k_400k'],
        'v400k_750k': ['v400k_500k', 'v500k_750k'],
        'v750k_plus': ['v750k_1m', 'v_1m_plus'],
    })
)
head(df)

48 cols x 512 rows


,year,county,med_age,households,avghhsize,civ_lf,emp,unemp,hispanic,white_nh,...,b1990_1999,b1980_1989,b1970_1979,b1960_1969,b1950_1959,b1949_e,ps_uni,ps_below,tot_l18,pov_l18
0,2012,ARAPAHOE,35.7,223747.0,2.55,318041.0,292089.0,25952.0,105174.0,364766.0,...,33989.0,56011.0,62253.0,22258.0,16519.0,7165.0,568999.0,66945.0,144576.0,23054.0
1,2012,MINERAL,60.3,363.0,1.83,391.0,370.0,21.0,15.0,671.0,...,232.0,239.0,203.0,100.0,75.0,240.0,702.0,47.0,26.0,0.0
2,2012,MONTROSE,42.6,16732.0,2.41,20124.0,18110.0,2014.0,8037.0,31799.0,...,3750.0,2106.0,3581.0,1298.0,920.0,2333.0,40368.0,5565.0,9788.0,1927.0


#### Create nominal variables for housing price and housing age
- First, create a categorical variable whose values are the COLUMN NAME of the bin with the max value. For instance, if a given county has more houses in the `v50k_100k` range than any other range, the value at that row in the new column will be "v50k_100k"
- Next, create a nominal column from that categorical column, ordered so that a lower number means less desirable. So for prices, "v_l_50k" -> 1, and for year built, "b1939_e" -> 1

In [24]:
@Extend(pd.DataFrame)
class _:
    def add_ordinal(
        self,
        col: str,
        order: list,
        replace: bool = False
    ) -> pd.DataFrame:
        """
        Create ordinal col from existing categorical col. Pass an ascending list
        of categories. Example: Input ['A', 'B', 'C'] -> New column map: {'A': 1, 'B': 2, 'C': 3}
        """
        df = self.copy()
        new = self[col].map({k: i+1 for i, k in enumerate(order)})
        df = df.insert_at(col, f'{col}_ord', new)
        if replace:
            df = df.drop_cols(col)
        return df

    def add_binmax(
        self,
        name: str,
        cols: list,
        replace: bool = False
    ) -> pd.DataFrame:
        """
        Shorthand for df.idxmax(), but lets you choose location and replace given columns
        """
        df = self.copy()
        new = self[cols].idxmax(axis=1)
        df = df.insert_at(cols[0], name, new)
        if replace:
            df = df.drop_cols(cols)
        return df

In [25]:
blt_ascending = ['b1949_e','b1950_1959','b1960_1969','b1970_1979','b1980_1989','b1990_1999','b2000_2009']
prices_ascending = ['v_l_50k', 'v50k_150k', 'v150k_250k', 'v250k_400k', 'v400k_750k', 'v750k_plus']
df = (df
    .add_binmax('blt_freq_yr', blt_ascending)
    .add_ordinal('blt_freq_yr', blt_ascending)
    .add_binmax('hu_freq_val', prices_ascending)
    .add_ordinal('hu_freq_val', prices_ascending)
)

---

#### Rename columns in population and census data with a naming system that uses consistent prefixes to let us easily select groups and sub-groups of columns with a simple string match

In [26]:
rename_pop = pd.read_excel('column_renaming.xlsx', sheet_name='population_rename')

rename_dem = pd.read_excel('column_renaming.xlsx', sheet_name='census_rename')

display(rename_pop.fillna(''), rename_dem.fillna(''))

pop = pop_raw.rename_cols_from_df(rename_pop.dropna())
df = df.rename_cols_from_df(rename_dem.dropna())

,OLD,NEW
0,total,pop
1,,
2,male,gend_m
3,female,gend_f
4,,
5,over18,age_over18
6,under19,age_undr19
7,,
8,over18_male,gend_m_age_over18
9,over18_female,gend_f_age_over18


,OLD,NEW
0,med_age,age_median
1,,
2,per_cap_in,inc_per_cap
3,med_hh_inc,inc_hh_median
4,,
5,households,hh
6,avghhsize,hh_size_avg
7,,
8,pop25plus,hsgrad_pool
9,hsgrad_sc,hsgrad_graduated


## Merge population and census data

In [27]:
main = (pop
    .merge(df, on=INDEX)
    .move_col('age_median', 'age_over18')
)
head(main)

61 cols x 512 rows


,year,county,pop,gend_m,gend_f,age_median,age_over18,age_undr19,gend_m_age_undr19,gend_f_age_undr19,...,hu_blt_1970_1979,hu_blt_1960_1969,hu_blt_1950_1959,hu_blt_freq_yr_ord,hu_blt_freq_yr,hu_blt_lt_1950,ps_known,ps_below,ps_undr18_known,ps_undr18_below
0,2012,ADAMS,231571.0,487410.0,201960.0,32.4,162109.0,69462.0,162653.0,66249.0,...,30185.0,19615.0,20369.0,7,b2000_2009,6158.0,438171.0,62008.0,124375.0,25278.0
1,2012,ALAMOSA,7823.0,17115.0,6283.0,32.2,5622.0,2201.0,5748.0,2044.0,...,1405.0,654.0,591.0,1,b1949_e,1536.0,14622.0,3191.0,3817.0,758.0
2,2012,ARAPAHOE,292548.0,666719.0,233180.0,35.7,212207.0,80341.0,227254.0,76419.0,...,62253.0,22258.0,16519.0,4,b1970_1979,7165.0,568999.0,66945.0,144576.0,23054.0


## Calculations for groups
---

In [28]:
# GroupedDF defined in workspace setup
GroupedDF.default_index = INDEX
GroupedDF.set_groups(['age', 'gend', 'race', 'inc', 'hh', 'citz', 'hsgrad', 'civ_lf', 'ps', 'stud', 'hu', 'hu_blt', 'hu_oo'])

In [29]:
gd = GroupedDF(main, INDEX, custom={'hu': INDEX + ['hu', 'hu_occ']})
# gd.df
# gd.display(3)

## Calculations
---

- **age, and gend**
  - `age_median`: (Existing)
  - `age_undr19_prop`: What percent of the population is under 19?
  - `gend_m_prop`: What percent of the population is male?
  - `age_undr19_gend_m_prop`: What percent of under-19 year old are male? (divide m_undr19 by undr19)
- **inc**
  - `inc_hh_med`: (Existing) Median household income
  - `inc_per_cap`: (Existing) Per capita income
- **hh**
  - `hh_size_avg`: (Existing) Average household size
- **race**
  - `race_{x}_prop`: What percent of the population is race x?
  - `race_prop_stdev`: What is the standard deviation of the race proportions? We need to calculate the proportions first, to normalize for the population size, that way, we can compare the standard deviations across groups
- **hsgrad**
  - `hsgrad_graduated_prop`: What percent of adults (age 25+) have a high school diploma or equivalent?
- **civ_lf**
  - `civ_lf_prop`: What percent of the population is in the civilian labor force?
  - `civ_lf_employed_prop`: What percent of the civilian labor force is employed?
- **ps**
  - `ps_total_prop`: What percent of people whose poverty status is known are below the poverty line?
  - `ps_undr18_total_prop`: What percent of under-18 people whose poverty status is known are below the poverty line?
  - `ps_undr18_prop`: What percent of people below the poverty line are under 18?
- **stud**
  - `stud_enrolled_prop`: Percent of people who could be enrolled in school that actually are enrolled
  - `stud_hs_prop`: What percent of gradeschool students (1-12) are high schoolers? (lower number indicates dropouts, which may associate with crime)
  - `stud_undergrad_prop`: What percent of enrolled students are undergraduates?
- **citz**
  - `citz_prop`: What percent of the population is a us citizen?
  - `citz_birth_prop`: What percent of us citizens were born in the us?
  - `citz_co_prop`: What percent of citizens were born in Colorado?
- **hu**
  - `hu_occ_prop`: Percent of homes which are occupied
  - `hu_blt_after1989`: Percent of homes which were built in the past 20 years
  - `hu_blt_nominal`: Convert hu_blt_mode_range to nominal, where the highest number corresponds to highest year range
- **hu_oo**
  - `hu_oo_prop`: Percent of occupied properties occupied by owner. The remaining percent is renter occupied
  - `hu_oo_lt_50_prop`: Percent of owner occupied properties worth less than $50,000
  - `hu_oo_750_plus_prop`: Percent of owner occupied properties worth $750,000 or more


In [30]:
df = main.copy()

df['age_over18_prop'] = df.age_over18 / df['pop']
df['age_undr19_prop'] = df.age_undr19 / df['pop']
df['gend_m_prop'] = df.gend_m / df['pop']
df['gend_f_prop'] = df.gend_f / df['pop']
df['age_undr19_gend_m_prop'] = df.gend_m_age_undr19 / df.age_undr19
df['age_undr19_gend_f_prop'] = df.gend_f_age_undr19 / df.age_undr19
df['age_over18_gend_m_prop'] = df.gend_m_age_over18 / df.age_over18
df['age_over18_gend_f_prop'] = df.gend_f_age_over18 / df.age_over18

df['gend_m_age_undr19_prop'] = df.gend_m_age_undr19 / df.gend_m
df['gend_m_age_over18_prop'] = df.gend_m_age_over18 / df.gend_m
df['gend_f_age_undr19_prop'] = df.gend_f_age_undr19 / df.gend_f
df['gend_f_age_over18_prop'] = df.gend_f_age_over18 / df.gend_f

race_base = GroupedDF(df, INDEX).race
race = df.copy()[INDEX]
for c in [c for c in race_base.columns if c not in INDEX]:
    race[f'{c}_prop'] = race_base[c] / df['pop']

race['race_prop_stdev'] = np.std(race.drop(columns=INDEX), axis=1)
df = df.merge(race, how='inner', on=INDEX)

df['hsgrad_graduated_prop'] = df.hsgrad_graduated / df.hsgrad_pool

df['civ_lf_prop'] = df.civ_lf / df['pop']
df['civ_lf_employed_prop'] = df.civ_lf_employed / df.civ_lf

df['ps_total_prop'] = df.ps_below / df.ps_known
df['ps_undr18_total_prop'] = df.ps_undr18_below / df.ps_undr18_known
df['ps_undr18_prop'] = df.ps_undr18_below / df.ps_below

df['stud_enrolled_prop'] = df.stud_enrolled / df.stud_enroll_pool
df['stud_hs_prop'] = df.stud_9_12 / (df.stud_1_4 + df.stud_5_8 + df.stud_9_12)
df['stud_undergrad_prop'] = df.stud_undergrad / df.stud_enrolled

df['citz_per_cap'] = df.citz / df['pop']
df['citz_birth_prop'] = df.citz_birth / df.citz
df['citz_co_prop'] = df.citz_co / df.citz

df['hu_per_cap'] = df.hu / df['pop']
df['hu_occ_prop'] = df.hu_occ / df.hu
df['hu_blt_2000_plus_prop'] = df.hu_blt_2000_plus / df.hu

df['hu_oo_prop'] = df.hu_oo / df.hu_occ

for hval in ['hu_oo_lt_50', 'hu_oo_50_150', 'hu_oo_150_250', 'hu_oo_250_400', 'hu_oo_400_750', 'hu_oo_750_plus']:
    df[f'{hval}_prop'] = df[hval] / df.hu_oo

for hyear in [
        'hu_blt_lt_1950', 'hu_blt_1950_1959', 'hu_blt_1960_1969',
        'hu_blt_1970_1979', 'hu_blt_1980_1989', 'hu_blt_1990_1999', 'hu_blt_2000_plus'
    ]:
    df[f'{hyear}_prop'] = df[hyear] / df.hu

# prop, counts = separate_by(df, ['prop', 'per_cap', 'median', 'avg', 'freq', 'med_hm_val', 'med_yr_blt'], index=INDEX)
filter_cols = ['prop', 'per_cap', 'median', 'avg', 'freq', 'med_hm_val', 'med_yr_blt']
prop = df.separate_by(filter_cols, index=INDEX, mode='include')
counts = df.separate_by(filter_cols, index=INDEX, mode='exclude')

gprop = GroupedDF(prop, INDEX, custom={'hu': INDEX + ['hu_per_cap', 'hu_occ_prop']})
# gprop.display()

In [31]:
write_main(prop, 'county_stats_normalized')
write_main(counts, 'county_stats_counts')
write_main(df, 'county_stats')

512